# Final Code for Scientific Project (Crew Rostering) - **Initial Solution**

In [ ]:
#Monday_data
#Step 1: Loading the Datasets
#First, we load both the railway data and the conductor availability data using Pandas.


import numpy as np
import pandas as pd
from collections import defaultdict

# Read data from Excel to dataframe
excel_file_path = 'weekly_trips_by_day_adjusted.xlsx'
monday_data = pd.read_excel(excel_file_path)

# Constants
MAX_WORKING_MINUTES = 600

# Conductor availability at each station
conductor_availability = {
    'A': 5,
    'B': 0,
    'C': 4,
    'D': 0,
    'E': 3,
    'F': 6,
    'G': 3,
    'H': 0,
    'I': 0,
    'J': 0,
    'K': 0,
    'L': 0
}

# Helper function to generate conductor names
def generate_conductor_name(station, count):
    return f"{count + 1}{station}"

# Function to calculate trip duration
def calculate_trip_duration(departure_time, arrival_time):
    return arrival_time - departure_time if arrival_time > departure_time else (1440 - departure_time) + arrival_time

# Adding trip duration to the dataframe
monday_data['trip_duration'] = monday_data.apply(lambda row: calculate_trip_duration(row['departure_time'], row['arrival_time']), axis=1)

# Resetting the conductor assignments and working minutes
conductor_working_minutes = {generate_conductor_name(station, i): 0 for station in conductor_availability for i in range(conductor_availability[station])}
conductor_last_trip_end_time = {name: None for name in conductor_working_minutes.keys()}

# Function to find a suitable conductor
def find_suitable_conductor(departure_station, trip_duration):
    for i in range(conductor_availability[departure_station]):
        conductor_name = generate_conductor_name(departure_station, i)
        if conductor_working_minutes[conductor_name] + trip_duration <= MAX_WORKING_MINUTES:
            return conductor_name
    return None

# Assigning conductors to trips based on the new rules
for train_number, trips in trips_by_train:
    last_conductor = None
    for _, trip in trips.iterrows():
        if trip['attendance_rate'] == 0:  # No conductor needed for 0% attendance rate
            continue

        # Find a suitable conductor
        conductor_name = find_suitable_conductor(trip['departure_station'], trip['trip_duration'])

        # If no conductor available at the departure station, check if the last conductor can continue
        if conductor_name is None and last_conductor and conductor_working_minutes[last_conductor] + trip['trip_duration'] <= MAX_WORKING_MINUTES:
            conductor_name = last_conductor

        if conductor_name:
            # Update conductor's working minutes
            conductor_working_minutes[conductor_name] += trip['trip_duration']

            # Assign conductor to the trip
            conductor_assignments[train_number].append({
                'trip_id': trip['id'],
                'conductor': conductor_name,
                'attendance_rate': trip['attendance_rate'],
                'departure_station': trip['departure_station'],
                'arrival_station': trip['arrival_station']
            })
            last_conductor = conductor_name  # Update last assigned conductor

# Populating the 'assigned_conductor' column in the original data with the new assignments
monday_data['assigned_conductor'] = np.nan  # Resetting the column
for train_number, assignments in conductor_assignments.items():
    for assignment in assignments:
        trip_id = assignment['trip_id']
        conductor_name = assignment['conductor']
        monday_data.loc[monday_data['id'] == trip_id, 'assigned_conductor'] = conductor_name

# Creating the new Excel file with the updated Monday data
new_monday_report_file_path = 'monday_trips_with_updated123_conductors.xlsx'
monday_data.to_excel(new_monday_report_file_path, index=False)


# Code for Heuristic
### Based on the shifted employees, the previous code was re-run to export the updated assignment of conductors

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

# Read data from Excel to dataframe
excel_file_path = 'monday_data.xlsx'
monday_data = pd.read_excel(excel_file_path)

# Constants
MAX_WORKING_MINUTES = 600

# Conductor availability at each station
conductor_availability = {
    'A': 7,
    'B': 0,
    'C': 3,
    'D': 0,
    'E': 3,
    'F': 6,
    'G': 2,
    'H': 0,
    'I': 0,
    'J': 0,
    'K': 0,
    'L': 0
}

# Helper function to generate conductor names
def generate_conductor_name(station, count):
    return f"{count + 1}{station}"

# Function to calculate trip duration
def calculate_trip_duration(departure_time, arrival_time):
    return arrival_time - departure_time if arrival_time > departure_time else (1440 - departure_time) + arrival_time

# Adding trip duration to the dataframe
monday_data['trip_duration'] = monday_data.apply(lambda row: calculate_trip_duration(row['departure_time'], row['arrival_time']), axis=1)

# Resetting the conductor assignments and working minutes
conductor_working_minutes = {generate_conductor_name(station, i): 0 for station in conductor_availability for i in range(conductor_availability[station])}
conductor_last_trip_end_time = {name: None for name in conductor_working_minutes.keys()}

# Function to find a suitable conductor
def find_suitable_conductor(departure_station, trip_duration):
    for i in range(conductor_availability[departure_station]):
        conductor_name = generate_conductor_name(departure_station, i)
        if conductor_working_minutes[conductor_name] + trip_duration <= MAX_WORKING_MINUTES:
            return conductor_name
    return None

# Grouping the trips by train_number
trips_by_train = monday_data.groupby('train_number')
trips_by_train = {train_number: trips for train_number, trips in trips_by_train}

# Initializing the conductor assignments dictionary
conductor_assignments = defaultdict(list)

# Assigning conductors to trips based on the new rules
for train_number, trips in trips_by_train.items():
    last_conductor = None
    for _, trip in trips.iterrows():
        if trip['attendance_rate'] == 0:  # No conductor needed for 0% attendance rate
            continue

        # Find a suitable conductor
        conductor_name = find_suitable_conductor(trip['departure_station'], trip['trip_duration'])

        # If no conductor available at the departure station, check if the last conductor can continue
        if conductor_name is None and last_conductor and conductor_working_minutes[last_conductor] + trip['trip_duration'] <= MAX_WORKING_MINUTES:
            conductor_name = last_conductor

        if conductor_name:
            # Update conductor's working minutes
            conductor_working_minutes[conductor_name] += trip['trip_duration']

            # Assign conductor to the trip
            conductor_assignments[train_number].append({
                'trip_id': trip['id'],
                'conductor': conductor_name,
                'attendance_rate': trip['attendance_rate'],
                'departure_station': trip['departure_station'],
                'arrival_station': trip['arrival_station']
            })
            last_conductor = conductor_name  # Update last assigned conductor

# Populating the 'assigned_conductor' column in the original data with the new assignments
monday_data['assigned_conductor'] = np.nan  # Resetting the column
for train_number, assignments in conductor_assignments.items():
    for assignment in assignments:
        trip_id = assignment['trip_id']
        conductor_name = assignment['conductor']
        monday_data.loc[monday_data['id'] == trip_id, 'assigned_conductor'] = conductor_name

# Creating the new Excel file with the updated Monday data
new_monday_report_file_path = 'iteration_5_variation_17.xlsx'
monday_data.to_excel(new_monday_report_file_path, index=False)

### Rough Work (Trying OOP for DB Crew Rostering)

#####The following code was only rough work to try attempting OOP in the problem, however the scope was changed and it was discarded.

In [ ]:
# Part 1

import pandas as pd


class Trip:
    def __init__(self, departure_station,
                 arrival_station,
                 departure_time,
                 arrival_time,
                 travel_time,
                 train_number,
                 attendance_rate,
                 days,
                 available_conductors):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.travel_time = travel_time
        self.train_number = train_number
        self.attendance_rate = attendance_rate
        self.days = days
        self.available_conductors = available_conductors
        self.assigned_conductor = None

    def assign_conductor(self, conductor):
        self.assigned_conductor = conductor


    def __repr__(self):
        return f"Trip(departure_station={self.departure_station}, arrival_station={self.arrival_station}, " \
               f"departure_time={self.departure_time}, arrival_time={self.arrival_time}, travel_time={self.travel_time},train_number={self.train_number}, " \
               f"attendance_rate={self.attendance_rate}, days={self.days}, available_conductors={self.available_conductors},assigned_conductor={self.assigned_conductor})"



# Read data from Excel to dataframe
excel_file_path = 'allday_railway_Mon.xlsx'
df = pd.read_excel(excel_file_path)

# Create instances of the Trip class from dataframe rows
trips = []
for index, row in df.iterrows():
    # Split the 'days' string into a list
    days_list = row['days'].split(', ')

    for day in days_list:

       trip = Trip(
          departure_station=row['departure_station'],
          arrival_station=row['arrival_station'],
          departure_time=row['departure_time'],
          arrival_time=row['arrival_time'],
          travel_time=row['arrival_time'] - row['departure_time'],
          train_number=row['train_number'],
          attendance_rate=row['attendance_rate'],
          days= day,
          available_conductors=[row['available_conductors']])
    trips.append(trip)

for trip in trips:
 print(trip)



Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, travel_time=7,train_number=1, attendance_rate=0.0, days=Mo, available_conductors=[5],assigned_conductor=None)
Trip(departure_station=C, arrival_station=D, departure_time=260, arrival_time=271, travel_time=11,train_number=2, attendance_rate=0.5, days=Mo, available_conductors=[4],assigned_conductor=None)
Trip(departure_station=E, arrival_station=F, departure_time=265, arrival_time=300, travel_time=35,train_number=3, attendance_rate=0.5, days=Mo, available_conductors=[3],assigned_conductor=None)
Trip(departure_station=G, arrival_station=F, departure_time=266, arrival_time=296, travel_time=30,train_number=4, attendance_rate=0.5, days=Mo, available_conductors=[3],assigned_conductor=None)
Trip(departure_station=D, arrival_station=G, departure_time=271, arrival_time=298, travel_time=27,train_number=2, attendance_rate=0.5, days=Mo, available_conductors=[0],assigned_conductor=None)
Trip(departure_station=F, arriv

In [ ]:
# Crew Rostering for DB Regio

import pandas as pd


class Trip:
    def __init__(self, departure_station,
                 arrival_station,
                 departure_time,
                 arrival_time,
                 travel_time,
                 train_number,
                 attendance_rate,
                 days,
                 available_conductors):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.travel_time = travel_time
        self.train_number = train_number
        self.attendance_rate = attendance_rate
        self.days = days
        self.available_conductors = available_conductors
        self.assigned_conductor = None

    def assign_conductor(self, conductor):
        self.assigned_conductor = conductor

    def assign_conductors(self):
        current_conductor_index = 0
        cumulative_travel_time = 0

        while cumulative_travel_time < self.travel_time:
            if self.available_conductors[current_conductor_index] > 0:
                self.assign_conductor(current_conductor_index)
                self.available_conductors[current_conductor_index] -= 1

                # Accumulate travel time for each assignment
                cumulative_travel_time += self.travel_time

                # Check if the accumulated travel time is within the specified range
                if cumulative_travel_time >= 550 and cumulative_travel_time <= 600:
                    current_conductor_index = (current_conductor_index + 1) % len(self.available_conductors)
                    cumulative_travel_time = 0
                elif self.arrival_station == self.departure_station:
                    current_conductor_index = (current_conductor_index + 1) % len(self.available_conductors)
                    cumulative_travel_time = 0
            else:
                current_conductor_index = (current_conductor_index + 1) % len(self.available_conductors)


    def __repr__(self):
        return f"Trip(departure_station={self.departure_station}, arrival_station={self.arrival_station}, " \
               f"departure_time={self.departure_time}, arrival_time={self.arrival_time}, travel_time={self.travel_time},train_number={self.train_number}, " \
               f"attendance_rate={self.attendance_rate}, days={self.days}, available_conductors={self.available_conductors},assigned_conductor={self.assigned_conductor})"



# Read data from Excel to dataframe
excel_file_path = 'allday_railway_Mon.xlsx'
df = pd.read_excel(excel_file_path)

# Create instances of the Trip class from dataframe rows
trips = []
for index, row in df.iterrows():
    # Split the 'days' string into a list
    days_list = row['days'].split(', ')

    for day in days_list:

       trip = Trip(
          departure_station=row['departure_station'],
          arrival_station=row['arrival_station'],
          departure_time=row['departure_time'],
          arrival_time=row['arrival_time'],
          travel_time=row['arrival_time'] - row['departure_time'],
          train_number=row['train_number'],
          attendance_rate=row['attendance_rate'],
          days= day,
          available_conductors=[row['available_conductors']])
    trip.assign_conductors() # assign conductors for the trip
    trips.append(trip)

for trip in trips:
 print(trip)



KeyboardInterrupt: ignored

Create Class for Conductor

In [ ]:
# make another class for available conductors

class Conductor:
    def __init__(self, stationName, crewBase, days, employees):
        self.stationName = stationName
        self.crewBase = crewBase
        # Handle NaN values
        if pd.notna(days):
            self.days = [str(x) for x in str(days).strip('[]').split(',')]
        else:
            self.days = []
          # Handle NaN values
        if pd.notna(employees):
            self.employees = [int(x) for x in str(employees).strip('[]').split(',')]
        else:
            self.employees = []

    def __repr__(self):
      return f"Conductor(stationName={self.stationName},crewBase={self.crewBase}, "\
             f"days={self.days}, conductors={self.employees}, "

# Read data from Excel to dataframe
excel_file_path = 'available_conductors.xlsx'
df = pd.read_excel(excel_file_path)

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Print actual column names
print(df.columns)


# Create instances of the Employee class from dataframe rows
conductors = []
for index, row in df.iterrows():
    # Check for NaN values in 'days' column
    if pd.notna(row['days']):
        days_list = row['days'].split(', ')
    else:
        days_list = []
    if pd.notna(row['employees']):
        employees_list = row['employees'].split(', ')
    else:
        employees_list = []

    # Iterate over days and employees, associating each conductor with a specific day
    for day, conductor_count in zip(days_list, employees_list):
        conductor = Conductor(row['name'], row['crewbase'], day, conductor_count)
        conductors.append(conductor)


for conductor in conductors:
  print(conductor)


Index(['name', 'crewbase', 'employees', 'days'], dtype='object')
Conductor(stationName=A,crewBase=True, days=['Mo'], conductors=[5], 
Conductor(stationName=A,crewBase=True, days=['Tu'], conductors=[5], 
Conductor(stationName=A,crewBase=True, days=['We'], conductors=[5], 
Conductor(stationName=A,crewBase=True, days=['Th'], conductors=[5], 
Conductor(stationName=A,crewBase=True, days=['Fr'], conductors=[5], 
Conductor(stationName=A,crewBase=True, days=['Sa'], conductors=[3], 
Conductor(stationName=A,crewBase=True, days=['Su'], conductors=[3], 
Conductor(stationName=B,crewBase=False, days=['Mo'], conductors=[0], 
Conductor(stationName=B,crewBase=False, days=['Tu'], conductors=[0], 
Conductor(stationName=B,crewBase=False, days=['We'], conductors=[0], 
Conductor(stationName=B,crewBase=False, days=['Th'], conductors=[0], 
Conductor(stationName=B,crewBase=False, days=['Fr'], conductors=[0], 
Conductor(stationName=B,crewBase=False, days=['Sa'], conductors=[0], 
Conductor(stationName=B,crewBase

Define Functions (Conductor Assignment,...)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

class Trip:
    def __init__(self, departure_station,
                 arrival_station,
                 departure_time,
                 arrival_time,
                 travel_time,
                 train_number,
                 attendance_rate,
                 days):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.travel_time = travel_time
        self.train_number = train_number
        self.attendance_rate = attendance_rate
        self.days = days
        self.assigned_conductor = None

    def assign_conductor(self, conductor):
        self.assigned_conductor = conductor

    def __repr__(self):
        return f"Trip(departure_station={self.departure_station}, arrival_station={self.arrival_station}, " \
               f"departure_time={self.departure_time}, arrival_time={self.arrival_time}, travel_time={self.travel_time},train_number={self.train_number}, " \
               f"attendance_rate={self.attendance_rate}, days={self.days}, assigned_conductor={self.assigned_conductor})"


class Conductor:
    def __init__(self, stationName, crewBase, days, employees):
        self.stationName = stationName
        self.crewBase = crewBase
        # Handle NaN values
        if pd.notna(days):
            self.days = [str(x) for x in str(days).strip('[]').split(',')]
        else:
            self.days = []
          # Handle NaN values
        if pd.notna(employees):
            self.employees = [int(x) for x in str(employees).strip('[]').split(',')]
        else:
            self.employees = []

    def __repr__(self):
      return f"Conductor(stationName={self.stationName},crewBase={self.crewBase}, "\
             f"days={self.days}, conductors={self.employees}, "


def assign_conductors_to_trips():
  for trip in trips:
    # find the available conductors from departing station on that day
    available_conductors = [conductor for conductor in conductors if
                            conductor.stationName == trip.departure_station and trip.days[0] in conductor.days and
                            conductor.employees > 0]

    if available_conductors:
      assigned_conductor = available_conductors.pop(0) # assigning the first available conductor
      assigned_conductor.employees -= 1 # Deduct 1 from available conductors
      trip.assign_conductor(assigned_conductor)



# Read data from Excel to dataframe
trip_excel_file_path = 'allday_railway.xlsx'
conductor_excel_file_path = 'available_conductors.xlsx'
trip_df = pd.read_excel(trip_excel_file_path)
conductor_df = pd.read_excel(conductor_excel_file_path)

# Create instances of the Trip class from dataframe rows
trips = []
for index, row in trip_df.iterrows():
    days_list = row['days'].split(', ')
    for day in days_list:
        trip = Trip(row['departure_station'], row['arrival_station'],
                    row['departure_time'], row['arrival_time'],
                    row['arrival_time'] - row['departure_time'], row['train_number'],
                    row['attendance_rate'], day)
        trips.append(trip)

# Create instances of the Conductor class from dataframe rows
conductors = []
for index, row in conductor_df.iterrows():
    if pd.notna(row['days']):
        days_list = row['days'].split(', ')
    else:
        days_list = []
    if pd.notna(row['employees']):
        employees_list = row['employees'].split(', ')
    else:
        employees_list = []

    for day, conductor_count in zip(days_list, employees_list):
        conductor = Conductor(row['name'], row['crewBase'], day, conductor_count)
        conductors.append(conductor)

# Assign conductors to trips
assign_conductors_to_trips()

# Print assigned trips
for trip in trips:
    print(trip)


Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, travel_time=7,train_number=1, attendance_rate=0.0, days=Mo, assigned_conductor=None)
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, travel_time=7,train_number=1, attendance_rate=0.0, days=Tu, assigned_conductor=None)
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, travel_time=7,train_number=1, attendance_rate=0.0, days=We, assigned_conductor=None)
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, travel_time=7,train_number=1, attendance_rate=0.0, days=Th, assigned_conductor=None)
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, travel_time=7,train_number=1, attendance_rate=0.0, days=Fr, assigned_conductor=None)
Trip(departure_station=C, arrival_station=D, departure_time=260, arrival_time=271, travel_time=11,train_number=2, attendance_rate=0.5, days=Mo, assigned_conduct

In [ ]:
class Employee:
    def __init__(self, stationName, crewBase, breakPossible, employees, partTimeEmployees):
        self.stationName = stationName
        self.crewBase = crewBase
        self.breakPossible = breakPossible
        self.employees = employees
        self.partTimeEmployees = partTimeEmployees

    def __repr__(self):
      return f"Employee(stationName={self.stationName},crewBase={self.crewBase}, "\
             f"breakPossible={self.breakPossible}, employees={self.employees}, "\
             f"partTimeEmployees={self.partTimeEmployees})"

# Read data from Excel to dataframe
excel_file_path = 'stations_2.xlsx'
df = pd.read_excel(excel_file_path)

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Print actual column names
print(df.columns)

# Create instances of the Employee class from dataframe rows
employees = []
for index, row in df.iterrows():
    employee = Employee(row['name'],
                        row['crewbase'],
                        row['breakpossible'],
                        row['employees'],
                        row['parttimeemployees'])
    employees.append(employee)


print(employees)


NameError: ignored

Building on to Shafaq's code. Following things to add

1. Conductors time limit (10 hours)
2. Move to next conductor after attendance rate is reached
3. Visualization of conductor assignment

In [ ]:
#assignment of conductors without considering attendance rate
import pandas as pd

class Trip:
    def __init__(self, departure_station, arrival_station, departure_time, arrival_time, train_number, attendance_rate, days, conductors_available):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.train_number = train_number
        self.attendance_rate = attendance_rate
        self.days = days
        self.conductors_available = str(conductors_available)  # Convert to string
        self.assigned_conductor = None

    def assign_conductor(self, conductor):
        self.assigned_conductor = conductor

    def __repr__(self):
        return f"Trip(departure_station={self.departure_station}, arrival_station={self.arrival_station}, " \
               f"departure_time={self.departure_time}, arrival_time={self.arrival_time}, train_number={self.train_number}, " \
               f"attendance_rate={self.attendance_rate}, days={self.days}, conductors_available={self.conductors_available}, assigned_conductor={self.assigned_conductor})"

class Conductor:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Conductor(name={self.name})"

def assign_conductors_to_trips(trips):
    for trip in trips:
        # Find available conductors for the departing station on the given day
        available_conductors = [Conductor(name) for name in trip.conductors_available.split(',')]

        # Assign the first available conductor to the trip
        if available_conductors:
            trip.assign_conductor(available_conductors[0])

# Read data from Excel to dataframe
excel_file_path = 'allday_railway_transformed.xlsx'
df = pd.read_excel(excel_file_path)

# Create instances of the Trip class from dataframe rows
trips = []
for index, row in df.iterrows():
    trip = Trip(row['departure_station'],
                row['arrival_station'],
                row['departure_time'],
                row['arrival_time'],
                row['train_number'],
                row['attendance_rate'],
                row['days'],
                row['conductors_available'])
    trips.append(trip)

# Assign conductors to trips based on available conductors at the departing station on that day
assign_conductors_to_trips(trips)

# Print the updated trips list with assigned conductors
for trip in trips:
    print(trip)

Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Mo, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=Conductor(name=1A))
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Tu, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=Conductor(name=1A))
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=We, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=Conductor(name=1A))
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Th, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=Conductor(name=1A))
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Fr, conductors_available=1A,2A,3A,4A,5A, assigned_condu

In [ ]:
# for journeys creation - easy to read data

import pandas as pd
import random

class Trip:
    def __init__(self, departure_station, arrival_station, departure_time, arrival_time, train_number, attendance_rate, days, conductors_available):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.train_number = train_number
        self.attendance_rate = attendance_rate
        self.days = days
        self.conductors_available = str(conductors_available)  # Convert to string
        self.assigned_conductor = None

    def assign_conductor(self, conductor):
        self.assigned_conductor = conductor

    def __repr__(self):
        return f"Trip(departure_station={self.departure_station}, arrival_station={self.arrival_station}, " \
               f"departure_time={self.departure_time}, arrival_time={self.arrival_time}, train_number={self.train_number}, " \
               f"attendance_rate={self.attendance_rate}, days={self.days}, conductors_available={self.conductors_available}, assigned_conductor={self.assigned_conductor})"

class Conductor:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Conductor(name={self.name})"

def assign_conductors_to_trips(trips):
    for trip in trips:
        # Check if a conductor should be assigned based on attendance rate
        if random.random() < trip.attendance_rate:
            # Find available conductors for the departing station on the given day
            available_conductors = [Conductor(name) for name in trip.conductors_available.split(',')]

            # Assign a random available conductor to the trip
            if available_conductors:
                trip.assign_conductor(random.choice(available_conductors))

def create_journeys(trips):
    journeys = []
    current_journey = []

    for trip in trips:
        if not current_journey or current_journey[-1].arrival_station == trip.departure_station:
            current_journey.append(trip)
        else:
            journeys.append(current_journey)
            current_journey = [trip]

    if current_journey:
        journeys.append(current_journey)

    return journeys

# Read data from Excel to dataframe
excel_file_path = 'allday_railway_transformed.xlsx'
df = pd.read_excel(excel_file_path)

# Create instances of the Trip class from dataframe rows and group them by train number and day
trips_by_train_and_day = {}
for index, row in df.iterrows():
    key = (row['train_number'], row['days'])
    if key not in trips_by_train_and_day:
        trips_by_train_and_day[key] = []

    trip = Trip(row['departure_station'],
                row['arrival_station'],
                row['departure_time'],
                row['arrival_time'],
                row['train_number'],
                row['attendance_rate'],
                row['days'],
                row['conductors_available'])
    trips_by_train_and_day[key].append(trip)

# Assign conductors to trips and create journeys for each train and day
for train_day_trips in trips_by_train_and_day.values():
    assign_conductors_to_trips(train_day_trips)
    train_day_journeys = create_journeys(train_day_trips)

    # Print the updated trips list with assigned conductors for each journey
    for journey in train_day_journeys:
        for trip in journey:
            print(trip)
        print('-' * 50)

Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Mo, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=None)
--------------------------------------------------
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Tu, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=None)
--------------------------------------------------
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=We, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=None)
--------------------------------------------------
Trip(departure_station=A, arrival_station=B, departure_time=233, arrival_time=240, train_number=1, attendance_rate=0.0, days=Th, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=None)
--------------------------------------------------
Trip(departure_station=A, arriva

In [ ]:
# assignment of conductors with respect to attendance rate
import pandas as pd
import random

class Trip:
    def __init__(self, departure_station, arrival_station, departure_time, arrival_time, train_number, attendance_rate, days, conductors_available):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.train_number = train_number
        self.attendance_rate = attendance_rate
        self.days = days
        self.conductors_available = str(conductors_available)  # Convert to string
        self.assigned_conductor = None

    def assign_conductor(self, conductor):
        self.assigned_conductor = conductor

    def __repr__(self):
        return f"Trip(departure_station={self.departure_station}, arrival_station={self.arrival_station}, " \
               f"departure_time={self.departure_time}, arrival_time={self.arrival_time}, train_number={self.train_number}, " \
               f"attendance_rate={self.attendance_rate}, days={self.days}, conductors_available={self.conductors_available}, assigned_conductor={self.assigned_conductor})"

class Conductor:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Conductor(name={self.name})"

class FeasibilityCheck:
    def __init__(self, train_number, total_trips, target_count):
        self.train_number = train_number
        self.total_trips = total_trips
        self.target_count = target_count
        self.assigned_count = 0

    def increment_assigned_count(self):
        self.assigned_count += 1

    def is_target_met(self):
        return self.assigned_count >= self.target_count

def assign_conductors_to_trips(trips):
    feasibility_checks = {}  # To keep track of attendance rate for each train
    for trip in trips:
        train_number = trip.train_number
        if train_number not in feasibility_checks:
            # Initialize a feasibility check object for the train
            total_trips = len([t for t in trips if t.train_number == train_number])
            target_count = total_trips // 2  # 50% of trips
            feasibility_checks[train_number] = FeasibilityCheck(train_number, total_trips, target_count)

        feasibility_check = feasibility_checks[train_number]

        # Check if the attendance rate is met for the train
        if not feasibility_check.is_target_met():
            # Find available conductors for the departing station on the given day
            available_conductors = [Conductor(name) for name in trip.conductors_available.split(',') if name != '0']

            # Assign a conductor based on availability or use the previous conductor
            if available_conductors:
                assigned_conductor = available_conductors[0]  # Select the first available conductor
                trip.assign_conductor(assigned_conductor)
                feasibility_check.increment_assigned_count()

            elif feasibility_check.assigned_count < feasibility_check.target_count:
                # If attendance rate is not met and no conductor is available
                print(f"No conductor available for Train {train_number} on Trip {trip.train_number}-{trip.days}.")

        # Check if conductor should continue the journey
        if trip.assigned_conductor:
            departure_time = pd.to_datetime(trip.departure_time)
            arrival_time = pd.to_datetime(trip.arrival_time)

            # Check if the total difference between departure and arrival time is <= 600
            if (arrival_time - departure_time).total_seconds() <= 600:
                # Continue the conductor to the arrival station
                next_trip = next((t for t in trips if t.train_number == train_number and t.days == trip.days and t.departure_station == trip.arrival_station), None)
                if next_trip:
                    next_trip.assign_conductor(trip.assigned_conductor)

# Read data from Excel to dataframe
excel_file_path = 'allday_railway_transformed.xlsx'
df = pd.read_excel(excel_file_path)

# Create instances of the Trip class from dataframe rows and group them by train number and day
trips_by_train_and_day = {}
for index, row in df.iterrows():
    key = (row['train_number'], row['days'])
    if key not in trips_by_train_and_day:
        trips_by_train_and_day[key] = []

    trip = Trip(row['departure_station'],
                row['arrival_station'],
                row['departure_time'],
                row['arrival_time'],
                row['train_number'],
                row['attendance_rate'],
                row['days'],
                row['conductors_available'])
    trips_by_train_and_day[key].append(trip)

# Assign conductors to trips for each train and day
for train_day_trips in trips_by_train_and_day.values():
    assign_conductors_to_trips(train_day_trips)

    # Print the updated trips list with assigned conductors for each journey
    for trip in train_day_trips:
        print(trip)
    print('-' * 50)

Streaming output truncated to the last 5000 lines.
No conductor available for Train 44 on Trip 44-Mo.
No conductor available for Train 44 on Trip 44-Mo.
No conductor available for Train 44 on Trip 44-Mo.
No conductor available for Train 44 on Trip 44-Mo.
Trip(departure_station=L, arrival_station=A, departure_time=517, arrival_time=533, train_number=44, attendance_rate=0.5, days=Mo, conductors_available=0, assigned_conductor=None)
Trip(departure_station=A, arrival_station=I, departure_time=541, arrival_time=549, train_number=44, attendance_rate=0.5, days=Mo, conductors_available=1A,2A,3A,4A,5A, assigned_conductor=Conductor(name=1A))
Trip(departure_station=I, arrival_station=K, departure_time=550, arrival_time=564, train_number=44, attendance_rate=0.5, days=Mo, conductors_available=0, assigned_conductor=Conductor(name=1A))
Trip(departure_station=K, arrival_station=J, departure_time=565, arrival_time=568, train_number=44, attendance_rate=0.5, days=Mo, conductors_available=0, assigned_cond